# [**Introduction à l'ingénierie automatisée des caractéristiques**](https://www.kaggle.com/code/willkoehrsen/introduction-to-manual-feature-engineering-p2/notebook)

408 votes

# Introduction : Bases de l'ingénierie automatisée des caractéristiques

Dans ce notebook, nous allons explorer l'application de l'ingénierie automatisée des caractéristiques sur l'ensemble de données [Home Credit Default Risk dataset](https://www.kaggle.com/c/home-credit-default-risk) en utilisant la bibliothèque featuretools. [Featuretools](https://docs.featuretools.com/) est une bibliothèque Python open-source qui permet de créer automatiquement de nouvelles caractéristiques à partir de plusieurs tables de données structurées et liées. C'est un outil idéal pour des problèmes tels que la compétition Home Credit Default Risk, où plusieurs tables liées doivent être combinées en un seul dataframe pour l'entraînement (et un pour les tests).

## Ingénierie des caractéristiques

L'objectif de l'[ingénierie des caractéristiques](https://en.wikipedia.org/wiki/Feature_engineering) est de créer de nouvelles caractéristiques (également appelées variables explicatives ou prédicteurs) afin de représenter autant d'informations que possible d'un ensemble de données complet dans une seule table. Habituellement, ce processus est effectué manuellement en utilisant des opérations pandas telles que `groupby`, `agg` ou `merge`, ce qui peut être très fastidieux. De plus, l'ingénierie des caractéristiques manuelle est limitée à la fois par les contraintes de temps humain et par l'imagination : nous ne pouvons tout simplement pas concevoir toutes les caractéristiques possibles qui seront utiles. (Pour un exemple d'utilisation de l'ingénierie des caractéristiques manuelle, consultez [partie 1](https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering) et [partie 2](https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering-p2) appliquées à cette compétition). L'importance de créer les bonnes caractéristiques ne peut être surestimée car un modèle d'apprentissage automatique ne peut apprendre que des données que nous lui fournissons. Extraire autant d'informations que possible des ensembles de données disponibles est crucial pour créer une solution efficace.

L'objectif de l'[ingénierie automatisée des caractéristiques](https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219) est d'aider le data scientist dans le processus de création des caractéristiques en construisant automatiquement des centaines voire des milliers de nouvelles caractéristiques à partir d'un ensemble de données. Featuretools, la seule bibliothèque d'ingénierie automatisée des caractéristiques disponible actuellement, ne vise pas à remplacer le data scientist, mais à lui permettre de se concentrer sur les parties les plus précieuses du processus d'apprentissage automatique, telles que la création de modèles robustes en production.

Ici, nous aborderons les concepts de l'ingénierie automatisée des caractéristiques avec featuretools et montrerons comment l'implémenter pour la compétition Home Credit Default Risk. Nous nous limiterons aux bases afin de bien comprendre les idées, et nous développerons ces bases dans des travaux ultérieurs lorsque nous personnaliserons featuretools. Nous travaillerons avec un sous-ensemble des données, car cette tâche est très intensive en termes de calcul et dépasse les capacités des noyaux Kaggle. J'ai utilisé le travail effectué dans ce notebook et exécuté les méthodes sur l'ensemble complet des données, les résultats étant [disponibles ici](https://www.kaggle.com/willkoehrsen/home-credit-default-risk-feature-tools). À la fin de ce notebook, nous examinerons les caractéristiques elles-mêmes, ainsi que les résultats de la modélisation avec différentes combinaisons de caractéristiques conçues manuellement et automatiquement construites.

Si vous débutez dans cette compétition, je vous suggère de consulter [ce billet pour commencer](https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction). Pour une bonne approche sur l'importance des caractéristiques, voici un [article de blog](https://www.featurelabs.com/blog/secret-to-data-science-success/) écrit par l'un des développeurs de Featuretools.

In [ ]:
# Uncomment and run if kernel does not already have featuretools
# !pip install featuretools

In [1]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# matplotlib and seaborn for visualizations
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 22
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

c:\Users\franc\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\pandas\__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


2023-06-28 12:24:13,934 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "DiversityScore" from "premium_primitives.diversity_score" because a primitive with that name already exists in "nlp_primitives.diversity_score"
2023-06-28 12:24:13,935 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "LSA" from "premium_primitives.lsa" because a primitive with that name already exists in "nlp_primitives.lsa"
2023-06-28 12:24:13,937 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "MeanCharactersPerSentence" from "premium_primitives.mean_characters_per_sentence" because a primitive with that name already exists in "nlp_primitives.mean_characters_per_sentence"
2023-06-28 12:24:13,937 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "NumberOfSentences" from "premium_primi

# Problème

La compétition Home Credit Default Risk est une tâche d'apprentissage automatique supervisée de classification. L'objectif est d'utiliser des données historiques sur la situation financière et socio-économique pour prédire si un demandeur sera en mesure de rembourser un prêt ou non. Il s'agit d'une tâche standard de classification supervisée :

* __Supervisée__ : Les étiquettes sont incluses dans les données d'entraînement et l'objectif est d'entraîner un modèle capable de prédire les étiquettes à partir des caractéristiques.
* __Classification__ : La variable cible est une variable binaire, 0 (remboursera le prêt à temps), 1 (rencontrera des difficultés à rembourser le prêt).

## Ensemble de données

Les données sont fournies par [Home Credit](http://www.homecredit.net/about-us.aspx), un service dédié à l'octroi de lignes de crédit (prêts) à la population non bancarisée.

Il existe 7 fichiers de données différents :

* __application_train/application_test__ : les données principales d'entraînement et de test contenant des informations sur chaque demande de prêt auprès de Home Credit. Chaque prêt a sa propre ligne et est identifié par `SK_ID_CURR`. Les données de demande d'entraînement sont accompagnées de la variable `TARGET` qui indique 0 : le prêt a été remboursé et 1 : le prêt n'a pas été remboursé.
* __bureau__ : données concernant les crédits précédents des clients provenant d'autres institutions financières. Chaque crédit précédent a sa propre ligne dans le bureau et est identifié par `SK_ID_BUREAU`. Chaque prêt dans les données de demande peut avoir plusieurs crédits précédents.
* __bureau_balance__ : données mensuelles sur les crédits précédents du bureau. Chaque ligne correspond à un mois d'un crédit précédent, et un crédit précédent unique peut avoir plusieurs lignes, une pour chaque mois de la durée du crédit.
* __previous_application__ : demandes précédentes de prêts chez Home Credit de clients ayant des prêts dans les données de demande. Chaque prêt en cours dans les données de demande peut avoir plusieurs prêts précédents. Chaque demande précédente a une ligne et est identifiée par la caractéristique `SK_ID_PREV`.
* __POS_CASH_BALANCE__ : données mensuelles sur les prêts précédents en points de vente ou en espèces que les clients ont eus avec Home Credit. Chaque ligne correspond à un mois d'un prêt précédent en points de vente ou en espèces, et un prêt précédent unique peut avoir de nombreuses lignes.
* __credit_card_balance__ : données mensuelles sur les anciennes cartes de crédit que les clients ont eues avec Home Credit. Chaque ligne correspond à un mois d'un solde de carte de crédit, et une seule carte de crédit peut avoir de nombreuses lignes.
* __installments_payment__ : historique des paiements pour les prêts précédents chez Home Credit. Il y a une ligne pour chaque paiement effectué et une ligne pour chaque paiement manqué.

Le diagramme ci-dessous (fourni par Home Credit) montre comment les tables sont liées. Cela sera très utile lorsque nous devrons définir des relations dans featuretools.

![image](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

### Lire les données et créer de petits ensembles de données

Nous allons lire l'ensemble de données complet, le trier par `SK_ID_CURR` et ne conserver que les 1000 premières lignes pour rendre les calculs réalisables. Plus tard, nous pourrons le convertir en script et l'exécuter avec l'ensemble complet des données.

In [3]:
# Read in the datasets and limit to the first 1000 rows (sorted by SK_ID_CURR) 
# This allows us to actually see the results in a reasonable amount of time!
def load_table(filename, by):
    return (
        pd.read_csv(f"../../dataset/csv/{filename}")
        .sort_values(by)
        .reset_index(drop = True).loc[:1000, :]
    )
    

"""
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index(drop = True).loc[:1000, :]
bureau_balance = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv').sort_values('SK_ID_BUREAU').reset_index(drop = True).loc[:1000, :]
cash = pd.read_csv('../input/home-credit-default-risk/POS_CASH_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
credit = pd.read_csv('../input/home-credit-default-risk/credit_card_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
previous = pd.read_csv('../input/home-credit-default-risk/previous_application.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
installments = pd.read_csv('../input/home-credit-default-risk/installments_payments.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
"""
app_train = load_table("application_train.csv", "SK_ID_CURR")
app_test = load_table("application_test.csv", "SK_ID_CURR")
bureau = load_table("bureau.csv", ["SK_ID_CURR", "SK_ID_BUREAU"])
bureau_balance = load_table("bureau_balance.csv", "SK_ID_BUREAU")
cash = load_table("POS_CASH_balance.csv", ["SK_ID_CURR", "SK_ID_PREV"])
credit = load_table("credit_card_balance.csv", ["SK_ID_CURR", "SK_ID_PREV"])
previous = load_table("previous_application.csv", ["SK_ID_CURR", "SK_ID_PREV"])
installments = load_table("installments_payments.csv", ["SK_ID_CURR", "SK_ID_PREV"])

Nous allons joindre l'ensemble d'entraînement et l'ensemble de test ensemble, mais ajouter une colonne distincte identifiant l'ensemble. Cela est important car nous allons vouloir appliquer exactement les mêmes procédures à chaque ensemble de données. Il est préférable de les joindre ensemble et de les traiter comme un seul dataframe.

(Je ne suis pas sûr si cela permet une fuite de données dans l'ensemble d'entraînement et si ces opérations de création de caractéristiques doivent être appliquées séparément. Toute réflexion à ce sujet serait grandement appréciée !)

In [4]:
# Add identifying column
app_train["set"] = "train"
app_test["set"] = "test"
app_test["TARGET"] = np.nan

# Append the dataframes
app = app_train.append(app_test, ignore_index = True)

# Bases de Featuretools

[Featuretools](https://docs.featuretools.com/#minute-quick-start) est une bibliothèque Python open-source permettant de créer automatiquement des caractéristiques à partir d'un ensemble de tables liées en utilisant une technique appelée [synthèse de caractéristiques profondes (DFS)](https://dai.lids.mit.edu/wp-content/uploads/2017/10/DSAA_DSM_2015.pdf). L'ingénierie automatique des fonctionnalités, comme de nombreux sujets en apprentissage automatique, est un sujet complexe construit sur des idées plus simples. En passant par ces idées une par une, nous pouvons développer notre compréhension de featuretools, ce qui nous permettra ultérieurement d'en tirer le meilleur parti.

Nous aborderons quelques concepts en cours de route :

* [Entités et ensembles d'entités](https://docs.featuretools.com/loading_data/using_entitysets.html)
* [Relations entre les tables](https://docs.featuretools.com/loading_data/using_entitysets.html#adding-a-relationship)
* [Caractéristiques primitives](https://docs.featuretools.com/automated_feature_engineering/primitives.html) : agrégations et transformations
* [Synthèse de caractéristiques profondes](https://docs.featuretools.com/automated_feature_engineering/afe.html)

# Entités et ensembles d'entités

Une entité est simplement une table ou, dans Pandas, un `dataframe`. Les observations se trouvent dans les lignes et les caractéristiques dans les colonnes. Une entité dans featuretools doit avoir un index unique où aucun des éléments n'est dupliqué. Actuellement, seuls `app`, `bureau` et `previous` ont des indices uniques (`SK_ID_CURR`, `SK_ID_BUREAU` et `SK_ID_PREV` respectivement). Pour les autres dataframes, nous devons passer `make_index=True` et spécifier le nom de l'index. Les entités peuvent également avoir des indices temporels où chaque entrée est identifiée par un temps unique. (Il n'y a pas de datetimes dans les données, mais il existe des temps relatifs, exprimés en mois ou en jours, que nous pourrions envisager de traiter comme des variables temporelles).

Un [EntitySet](https://docs.featuretools.com/loading_data/using_entitysets.html) est un regroupement de tables et des relations entre elles. On peut le considérer comme une structure de données avec ses propres méthodes et attributs. Utiliser un ensemble d'entités nous permet de regrouper plusieurs tables et de les manipuler beaucoup plus rapidement que les tables individuelles.

Tout d'abord, nous allons créer un ensemble d'entités vide nommé "clients" pour suivre l'ensemble des données.

In [5]:
# Entity set with id applications
es = ft.EntitySet(id = 'clients')